In [1]:
import os
import numpy as np
import pandas as pd
import re
from scipy import stats

# from utils.logger import LOGPATH

In [8]:
# fpath = LOGPATH.joinpath('summary-fb.csv')
fpath = '/home/nliao/code/Spectral-GNN-Benchmark/log/summary-mb.csv'
t = pd.read_csv(fpath, engine='python')

'''
fpath = '/home/nliao/code/Spectral-GNN-Benchmark/log/summary-mb-f.csv'
t2 = pd.read_csv(fpath, engine='python')

t_id = t.apply(lambda row: '+'.join((str(row['seed']), row['data'], row['model'], row['conv'])), axis=1)
t2_id = t2.apply(lambda row: '+'.join((str(row['seed']), row['data'], row['model'], row['conv'])), axis=1)
t2_id.drop_duplicates(keep='last', inplace=True)
t2 = t2.loc[t2_id.index]

row_id = []
for string in t2_id:
    match_index = t[t_id == string].index
    row_id.append(match_index[0])
t2.index = row_id
t.update(t2)
'''

t['time_learn'] = t['time_learn'] * 1000 / t['epoch_learn']
t['time_eval'] = t['time_train'] + t['time_val'] + t['time_test']
t['time_eval'] = t['time_eval'] * 1000

t['s_test'] = t.apply(lambda row: row[row['metric'] + '_test'], axis=1)
# t['zscore'] = t.groupby(['data', 'model', 'conv'])['s_test'].transform(lambda x: stats.zscore(x, ddof=0))
t['median'] = t.groupby(['data', 'model', 'conv'])['s_test'].transform('median')
t['std'] = t.groupby(['data'])['s_test'].transform('std')
outlier = t['s_test'] < t['median'] - 2 * t['std']
t[outlier]

,seed,data,metric,model,conv,time_pre,mem_ram_pre,mem_cuda_pre,epoch_best,epoch_learn,...,s_ap_test,s_auroc_test,s_f1i_test,time_test,mem_ram_eval,mem_cuda_eval,time_eval,s_test,median,std
248,28,chameleon_filtered,s_f1i,PrecomputedFixed,AdjConv-gaussian,0.0362,1.301,0.028,10,500,...,32.724,62.001,29.775,0.0003,1.301,0.028,0.9,29.775,38.7640,4.254534
728,21,citeseer,s_f1i,PrecomputedVar,ChebIIConv,0.5812,2.240,0.459,145,200,...,26.156,59.756,29.771,0.0073,2.240,0.450,22.1,29.771,72.5950,5.474821
731,24,citeseer,s_f1i,PrecomputedVar,ChebIIConv,0.6022,2.242,0.459,190,200,...,38.216,67.467,39.695,0.0074,2.242,0.450,22.4,39.695,72.5950,5.474821
734,27,citeseer,s_f1i,PrecomputedVar,ChebIIConv,0.6248,2.243,0.459,199,200,...,52.676,76.747,50.076,0.0072,2.243,0.450,22.8,50.076,72.5950,5.474821
736,29,citeseer,s_f1i,PrecomputedVar,ChebIIConv,0.5786,2.243,0.459,199,200,...,39.119,69.299,40.153,0.0073,2.243,0.450,22.1,40.153,72.5950,5.474821
836,26,flickr,s_f1i,PrecomputedVar,AdjConv,0.1375,2.146,0.231,1,200,...,20.000,50.000,11.250,0.0013,2.146,0.231,4.2,11.250,33.0260,4.390117
838,28,flickr,s_f1i,PrecomputedVar,AdjConv,0.1391,2.223,0.231,1,200,...,20.461,50.776,22.829,0.0011,2.223,0.231,4.6,22.829,33.0260,4.390117
900,20,flickr,s_f1i,PrecomputedVar,JacobiConv,0.1825,1.901,0.000,3,200,...,21.234,52.138,22.961,0.0008,1.901,0.231,3.5,22.961,33.7500,4.390117
998,28,chameleon_filtered,s_f1i,PrecomputedVar,LegendreConv,0.0577,1.420,0.095,53,200,...,29.995,60.318,28.652,0.0012,1.420,0.095,3.6,28.652,39.3260,4.254534
1057,27,squirrel_filtered,s_f1i,PrecomputedVar,ChebConv,0.1690,1.801,0.187,66,200,...,24.044,54.486,28.604,0.0009,1.801,0.187,3.7,28.604,34.2340,2.419963


In [9]:
dlst = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire", ]
        # "amazon_ratings", "minesweeper", "tolokers", "questions", "reddit", "penn94", \
        # "ogbn-arxiv", "arxiv-year", "genius", "twitch-gamer", "ogbn-mag"]
id_cols = ["seed", 'data', 'model', 'conv',]
m_cols = ['time_pre', 'mem_ram_pre', 'time_learn', 'mem_ram_train', 'mem_cuda_train', 's_test', 'time_eval',]
conv_repr = {
    'MLP': {'Identity-impulse': 'Identity',},
    'PrecomputedFixed': {
        'AdjSkipConv-ones': 'Linear',
        'AdjConv-impulse':  'Impulse',
        'AdjConv-mono':     'Monomial',
        'AdjConv-appr':     'PPR',
        'AdjConv-hk':       'HK',
        'AdjConv-gaussian': 'Gaussian',},
    'PrecomputedVar': {
        'AdjSkipConv':      'Var-Linear',
        'AdjConv':          'Var-Monomial',
        'HornerConv':       'Horner',
        'ChebConv':         'Chebyshev',
        'ClenshawConv':     'Clenshaw',
        'ChebIIConv':       'ChebInterp',
        'BernConv':         'Bernstein',
        'LegendreConv':     'Legendre',
        'JacobiConv':       'Jacobi',
        'OptBasisConv':     'OptBasis',},
    # 'AdaGNN': {'LapiConv':  'AdaGNN',},
        # 'OptBasisConv':     'OptBasisGNN',},
    'PrecomputedFixedCompose': {
        'AdjSkipConv,AdjSkipConv-ones,ones': 'FAGNN',
        'AdjSkip2Conv,AdjSkip2Conv-gaussian,gaussian': 'G$^2$CN',
        'AdjDiffConv,AdjDiffConv-appr,appr': 'GNN-LF/HF',},
    'PrecomputedVarCompose': {
        'AdjConv,ChebConv,BernConv': 'FiGURe',}}

def reverse_name(name):
    for k, m in conv_repr.items():
        for c, n in m.items():
            if n == name:
                return k, c
    return None, None

flst = []
for m in conv_repr.values():
    for c, name in m.items():
        flst.append(name)

nil_repr = []
for row in t.itertuples():
    if row.model not in conv_repr:
        nil_repr.append(row.model)
    else:
        if row.conv not in conv_repr[row.model]:
            nil_repr.append('+'.join((row.model, row.conv)))
set(nil_repr)

set()

In [10]:
def get_df(t):
    tt = t[t['data'].isin(dlst)][id_cols+m_cols].sort_values(by=['data', 'model', 'conv', 'seed'])
    # print(tt.sort_index(ascending=False).groupby(['data', 'model', 'conv']).filter(lambda x: len(x) > 10))
    df = tt.groupby(['data', 'model', 'conv'])[m_cols].agg(['mean', 'std'])
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace=True)

    df['name'] = df.apply(lambda row: conv_repr[row['model']][row['conv']] if (row['model'] in conv_repr) and (row['conv'] in conv_repr[row['model']]) else None, axis=1)
    df.dropna(subset=['name'], inplace=True)
    df.drop(columns=['model', 'conv'], inplace=True)
    df = df[['data', 'name'] + df.columns.tolist()[1:-1]]
    return df

# t = t[~outlier]
df = get_df(t)
df

,data,name,time_pre_mean,time_pre_std,mem_ram_pre_mean,mem_ram_pre_std,time_learn_mean,time_learn_std,mem_ram_train_mean,mem_ram_train_std,mem_cuda_train_mean,mem_cuda_train_std,s_test_mean,s_test_std,time_eval_mean,time_eval_std
0,actor,PPR,0.38011,0.314038,1.7189,0.260530,4.46050,0.675792,1.7512,0.159422,0.040,0.000000,32.4868,1.221574,1.51,0.233095
1,actor,Gaussian,0.08924,0.020007,1.7967,0.274220,4.49722,0.490596,1.8356,0.156197,0.040,0.000000,32.8421,1.279674,1.40,0.115470
2,actor,HK,0.11394,0.047949,1.6106,0.221253,4.44284,0.270204,1.6368,0.138631,0.040,0.000000,32.8816,1.392876,1.54,0.150555
3,actor,Impulse,0.45752,0.450204,1.6936,0.266873,4.67524,0.252514,1.7253,0.181391,0.040,0.000000,24.3159,1.604023,1.52,0.220101
4,actor,Monomial,0.11726,0.034471,1.7299,0.245460,4.58326,0.575605,1.7609,0.148057,0.040,0.000000,32.6381,1.409783,1.51,0.207900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,squirrel_filtered,Horner,0.12937,0.016261,1.6958,0.158014,4.22715,0.351658,1.7209,0.082082,0.199,0.000000,36.6217,1.689554,5.14,0.779173
160,squirrel_filtered,Jacobi,0.15543,0.020712,1.7987,0.207198,4.12935,0.376226,1.8267,0.121503,0.186,0.003162,35.8559,1.801267,3.31,0.570477
161,squirrel_filtered,Legendre,0.13262,0.008487,1.7870,0.161763,4.29080,0.511529,1.8047,0.107887,0.186,0.003162,36.1486,2.018017,3.35,0.690008
162,squirrel_filtered,OptBasis,2.28026,0.386258,1.8252,0.180559,4.37865,0.317319,1.8486,0.106727,0.186,0.003162,34.9999,1.624892,3.58,0.666333


## Fine

In [11]:
# dlst_acc = dlst
dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
# dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "reddit", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
# dlst_acc = ["ogbn-arxiv", "amazon_ratings", "minesweeper", "tolokers", "questions", "penn94", "arxiv-year", "genius", "twitch-gamer"]

acc_dual = pd.DataFrame(index=flst, columns=pd.MultiIndex.from_product([dlst_acc, ['mean', 'std']]), dtype=float)
acc_str = pd.DataFrame(index=flst, columns=dlst_acc, dtype=str)
for row in acc_str.index:
    for col in acc_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == col), 's_test_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == col), 's_test_std'].values
        if len(mean) > 0:
            acc_dual.loc[row, col] = mean[0], std[0]
            acc_str.loc[row, col] = f'{mean[0]:.2f}\\tpm{{{std[0]:.2f}}}'

acc_dual.style.background_gradient(axis=0)